<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Notebook-Description" data-toc-modified-id="Notebook-Description-0.0.1">Notebook Description</a></span></li><li><span><a href="#Importing-SBML-Model" data-toc-modified-id="Importing-SBML-Model-0.0.2">Importing SBML Model</a></span><ul class="toc-item"><li><span><a href="#Construct-SciPyModel-Instance" data-toc-modified-id="Construct-SciPyModel-Instance-0.0.2.1">Construct SciPyModel Instance</a></span></li><li><span><a href="#User-File-Selection" data-toc-modified-id="User-File-Selection-0.0.2.2">User File Selection</a></span></li><li><span><a href="#Clean-SBML-File" data-toc-modified-id="Clean-SBML-File-0.0.2.3">Clean SBML File</a></span></li><li><span><a href="#Extract-MetaData" data-toc-modified-id="Extract-MetaData-0.0.2.4">Extract MetaData</a></span></li><li><span><a href="#Extract-Compartment-Data" data-toc-modified-id="Extract-Compartment-Data-0.0.2.5">Extract Compartment Data</a></span></li><li><span><a href="#Extract-Species-Data" data-toc-modified-id="Extract-Species-Data-0.0.2.6">Extract Species Data</a></span></li><li><span><a href="#Extract-Parameter-Data" data-toc-modified-id="Extract-Parameter-Data-0.0.2.7">Extract Parameter Data</a></span></li><li><span><a href="#Extract-Reaction-Data" data-toc-modified-id="Extract-Reaction-Data-0.0.2.8">Extract Reaction Data</a></span></li><li><span><a href="#Vectorize-Reaction-Declarations" data-toc-modified-id="Vectorize-Reaction-Declarations-0.0.2.9">Vectorize Reaction Declarations</a></span></li><li><span><a href="#Order-Reactions-by-Parameter-Index" data-toc-modified-id="Order-Reactions-by-Parameter-Index-0.0.2.10">Order Reactions by Parameter Index</a></span></li></ul></li><li><span><a href="#Final-Notes" data-toc-modified-id="Final-Notes-0.0.3">Final Notes</a></span></li></ul></li></ul></li></ul></div>

### Notebook Description

This notebook walks through the way in which an SBML version model is converted into a custom class `SciPyModel`. The process includes reading/storing the model information, automatically writing a function for use with `odeint`, and simulating the internal model.

### Importing SBML Model

#### Construct SciPyModel Instance
The SciPyModel is a custom class that is created by calling the `createSciPyModel` command.

In [ ]:
import os, sys
NotebookDirectory = os.path.split(os.getcwd())[0]
if NotebookDirectory not in sys.path:
    sys.path.append(NotebookDirectory)
    
import toolbox 

SciPyModel = toolbox.createSciPyModel()

#### User File Selection

The `Tkinter` module provides a quick way to open a file browser for user selection. A conditional statement is used to check if a model has already been opened in this session.

In [ ]:
import Tkinter, tkFileDialog

if SciPyModel.MetaData.FilePath == None:
    Tkinter.Tk().withdraw()
    SciPyModel.MetaData.FilePath = tkFileDialog.askopenfilename()
else:
    pass

#### Clean SBML File

Now we can open the SBML document in the file and make edits to ensure uniformity of input.

In [ ]:
# Import required packages
import libsbml

# Read in SBML model file into SBML Document variable
SBMLDoc = libsbml.readSBMLFromFile(SciPyModel.MetaData.FilePath)

# Check if any major errors in reading SBML model
# e.g. Filepath does not exist
if SBMLDoc.getNumErrors() > 0:
    print('ERROR: File reading errors.')
    print(SBMLDoc.getErrorLog().toString())

# Make all parameters of the model global parameters.
# Enables all parameters to be vectorized.
Properties = libsbml.ConversionProperties()
Properties.addOption('promoteLocalParameters', True)
if SBMLDoc.convert(Properties) != libsbml.LIBSBML_OPERATION_SUCCESS:
    print('ERROR: Unable to convert parameters to global.')
    print(SBMLDoc.getErrorLog().toString())

# Write out all reaction-specific function definitions.
# Enables all variables in reactions to be swapped with vectorized
# versions.
Properties = libsbml.ConversionProperties()
Properties.addOption('expandFunctionDefinitions', True)
if SBMLDoc.convert(Properties) != libsbml.LIBSBML_OPERATION_SUCCESS:
    print('ERROR: Unable to expand internal function usage.')
    print(SBMLDoc.getErrorLog().toString())

# Write out all state variable and parameter initializations.
# Enables this data to be placed into required SciPyModel object
# places.
Properties = libsbml.ConversionProperties()
Properties.addOption('expandInitialAssignments', True)
if SBMLDoc.convert(Properties) != libsbml.LIBSBML_OPERATION_SUCCESS:
    print('ERROR: Unable to expand initial assignments.')
    print(SBMLDoc.getErrorLog().toString())

# Extract SBML Model object from SBML Document object.
SBMLModel = SBMLDoc.getModel()

#### Extract MetaData

In [ ]:
# -- Name, VolumeUnits, SubstanceUnits, TimeUnits
SciPyModel.MetaData.Name = SBMLModel.getName()
SciPyModel.MetaData.VolumeUnits = SBMLModel.getVolumeUnits()
SciPyModel.MetaData.SubstanceUnits = SBMLModel.getSubstanceUnits()
SciPyModel.MetaData.TimeUnits = SBMLModel.getTimeUnits()

#### Extract Compartment Data

In [ ]:
# -- Quantity, Names, VectorIndex
SciPyModel.Compartments.Quantity = SBMLModel.getNumCompartments()
for i in range(SBMLModel.getNumCompartments()):
    current_compartment = SBMLModel.getCompartment(i)
    SciPyModel.Compartments.Names.append(current_compartment.name)
    SciPyModel.Compartments.VectorIndex.append(i)
    SciPyModel.Compartments.MetaID.append(current_compartment.meta_id)

#### Extract Species Data

In [ ]:
# -- Quanity, Names, Value, VectorIndex, BoundaryValue
SciPyModel.Species.Quantity = SBMLModel.getNumSpecies()
for i in range(SBMLModel.getNumSpecies()):
    current_species = SBMLModel.getSpecies(i)
    SciPyModel.Species.Names.append(current_species.name)
    SciPyModel.Species.Value.append(current_species.initial_amount)
    SciPyModel.Species.VectorIndex.append(i)
    SciPyModel.Species.BoundaryValue.append(current_species.boundary_condition)
    SciPyModel.Species.MetaID.append(current_species.meta_id)

#### Extract Parameter Data

In [ ]:
# -- Quantity, Names, Value, VectorIndex
SciPyModel.Parameters.Quantity = SBMLModel.getNumParameters()
for i in range(SBMLModel.getNumParameters()):
    current_parameter = SBMLModel.getParameter(i)
    SciPyModel.Parameters.Names.append(current_parameter.name)
    SciPyModel.Parameters.Value.append(current_parameter.value)
    SciPyModel.Parameters.VectorIndex.append(i)
    SciPyModel.Parameters.MetaID.append(current_parameter.meta_id)

#### Extract Reaction Data

Requires use of `NumPy` module.

In [ ]:
import numpy

# -- Names, Formulas, Stoichiometry
SciPyModel.Reactions.Stoichiometry = numpy.zeros(
    [SBMLModel.getNumSpecies(),
     SBMLModel.getNumReactions()])
SciPyModel.Reactions.Quantity = SBMLModel.getNumReactions()
for i in range(SBMLModel.getNumReactions()):
    current_reaction = SBMLModel.getReaction(i)
    SciPyModel.Reactions.Names.append(current_reaction.name)
    SciPyModel.Reactions.Formulas.append(
        current_reaction.getKineticLaw().getFormula())

    # Try-Except in order to see if Names or MetaID are used in the functions
    try:
        for r in current_reaction.getListOfReactants():
            SciPyModel.Reactions.Stoichiometry[SciPyModel.Species.Names.index(
                r.getSpecies()), i] -= r.getStoichiometry()
        for p in current_reaction.getListOfProducts():
            SciPyModel.Reactions.Stoichiometry[SciPyModel.Species.Names.index(
                p.getSpecies()), i] += p.getStoichiometry()
    except ValueError:
        for r in current_reaction.getListOfReactants():
            SciPyModel.Reactions.Stoichiometry[SciPyModel.Species.MetaID.index(
                r.getSpecies()), i] -= r.getStoichiometry()
        for p in current_reaction.getListOfProducts():
            SciPyModel.Reactions.Stoichiometry[SciPyModel.Species.MetaID.index(
                p.getSpecies()), i] += p.getStoichiometry()
    else:
        print 'ERROR: Unable to create Stoichiometric Matrix. Check species name/metaid.'

    # Remove Stoichiometry of Boundary State Variables
    for s in range(SciPyModel.Species.Quantity):
        if SciPyModel.Species.BoundaryValue[s]:
            SciPyModel.Reactions.Stoichiometry[s, :] = numpy.zeros(
                (1, SciPyModel.Reactions.Quantity))

#### Vectorize Reaction Declarations

In [ ]:
# Vectorize Functions within SciPyModel object.
for rxn_ix in range(SciPyModel.Reactions.Quantity):

    # Get information about the current reaction
    Formula = SciPyModel.Reactions.Formulas[rxn_ix]
    
    # Removes usage of compartments from reaction equations.
    for j in reversed(range(SciPyModel.Compartments.Quantity)):
        if SciPyModel.Compartments.Names[j] != '':  # If name isn't empty
            Formula = Formula.replace(SciPyModel.Compartments.Names[j] + ' * ',
                                      '')
            Formula = Formula.replace(' * ' + SciPyModel.Compartments.Names[j],
                                      '')
            Formula = Formula.replace(' / ' + SciPyModel.Compartments.Names[j],
                                      '')
    
    # Replace parameter names with index of vectorized parameter array.
    # Iterates through parameter names sorted by length of name.
    for key in sorted(
            SciPyModel.Parameters.Names, key=len, reverse=True):
        if key != '':
            Formula = Formula.replace(key, 'p[' + str(
                SciPyModel.Parameters.Names.index(key)) + ']')

    for key in sorted(
            SciPyModel.Parameters.MetaID, key=len, reverse=True):
        if key != '':
            Formula = Formula.replace(key, 'p[' + str(
                SciPyModel.Parameters.MetaID.index(key)) + ']')
            
    # Replace species names with index of species parameter array.
    for key in sorted(SciPyModel.Species.Names, key=len, reverse=True):
        if key != '':
            Formula = Formula.replace(
                key, 'y[' + str(SciPyModel.Species.Names.index(key)) + ']')
            
    for key in sorted(SciPyModel.Species.MetaID, key=len, reverse=True):
        if key != '':
            Formula = Formula.replace(
                key, 'y[' + str(SciPyModel.Species.MetaID.index(key)) + ']')
            
    # Reset formula declaration in SciPyModel class
    SciPyModel.Reactions.Formulas[rxn_ix] = Formula

#### Order Reactions by Parameter Index

This is key to ensuring a proper deployment of the nullspace matrix for parameter sampling using a steady-state condition.

In [ ]:
# Grab indecies of kinetic rate constant parameters
ReactionIndex = []
for rxn_ix in range(SciPyModel.Reactions.Quantity):
    ReactionIndex.append(
        int(SciPyModel.Reactions.Formulas[rxn_ix].split(']')[0].split('[')[
            -1]))

# Order the parameters and track the internal index ordering
SortedIndex = [
    i[0] for i in sorted(enumerate(ReactionIndex), key=lambda x: x[1])
]

# Apply the index ordering to order the formula vector
SciPyModel.Reactions.Formulas = [
    SciPyModel.Reactions.Formulas[i] for i in SortedIndex
]

# Apply the index ordering to order the stoichiometry matrix
SciPyModel.Reactions.Stoichiometry = numpy.vstack([
    SciPyModel.Reactions.Stoichiometry[:, i] for i in SortedIndex
]).transpose()

### Final Notes

Once the SBML model has been imported, several analysis methods become readily available thought the `SBML-Analysis-Toolbox`. These functions can be called through the `toolbox` module.

Full discussion of these implementations can be found in their respective notebooks.